In [ ]:
import pandas as pd
import os
from datetime import datetime, timedelta
import sys
import psycopg2
from psycopg2.errors import UniqueViolation, StringDataRightTruncation

# Funções auxiliares

In [ ]:
def le_csv(path, dtype=None):
    '''
    Função para ler csvs, de acordo com a codificação
    '''

    print(path)
    header = open(path, 'r').readline()
    if '\t' in header:
        df = pd.read_csv(path, sep="\t", decimal=",", encoding = "utf-16", error_bad_lines=False, dtype=dtype, quoting=csv.QUOTE_NONE)
    elif ';' in header:
        df =  pd.read_csv(path, sep=";", decimal=",", encoding = "ansi", error_bad_lines=False, dtype=dtype, quoting=csv.QUOTE_NONE)
        print(f'Ponto vírgula: {path}')
    else: 
        print(f'Outro delim.: {path}')
        print(header)
        sys.exit()

    return df

In [ ]:
def check_insert(cursor, table, dict_select, dict_insert):
    """
    Checa se existe uma entrada com os pares coluna e valor do dicionário 'dict_select' na tabela 'table',
    se sim, retorna seu id, se não, cria uma entrada com os pares coluna e valor do dicionário 'dict_insert'
    e retorna o id dessa nova entrada
    """

    # Prepara molde da clausula WHERE e valores que receberá, para a query SELECT
    select_molde = f"({' AND '.join([col + ' = %s' for col in dict_select.keys()])})"
    select_values = tuple(val for val in dict_select.values())

    # Procura entrada na base
    cursor.execute(
        f"""
        SELECT id FROM {table} 
        WHERE {select_molde}
        """,
        select_values
    )
    result = cursor.fetchall()

    # Se a busca retornou algo
    if result:

        # Pega id do resultado retornado
        id_table = result[0][0] 

    # Se a busca não retornou nada (a entrada nao existe)
    else:
         
         # Prepara moldes das e valores que receberá da query INSERT
        insert_cols_molde = f"({', '.join(dict_insert.keys())})"
        insert_values_molde = f"({', '.join('%s' for i in dict_insert)})"
        insert_values = tuple(val for val in dict_insert.values())     

        #print(table, insert_cols_molde, insert_values_molde, insert_values)  

        # Insere a nova entrada na base
        try:
            cursor.execute("SAVEPOINT bulk_savepoint")
            cursor.execute(
                f"""
                INSERT INTO {table} {insert_cols_molde} VALUES {insert_values_molde}
                RETURNING id
                """,
                insert_values
            )
            
            # Pega id da linha inserida
            id_table = cursor.fetchone()[0]

        except StringDataRightTruncation as e:
            c.execute("ROLLBACK TO SAVEPOINT bulk_savepoint")
            print(e)
            print(dict_insert, '\n')
            id_table = None

        finally:
            c.execute("RELEASE SAVEPOINT bulk_savepoint")
            
    return id_table

# Pacientes

In [ ]:
# Le csv para pegar colunas e especificar tipo de cada
pacientes = pd.read_csv(r'D:\Banco_integracao\Base pastas separadas\Banco de dados Paciente.csv', sep="\t", decimal=",", encoding = "utf-16", dtype=str).iloc[:-4, :]
colunas = pacientes.columns
dict_dtypes = {x : 'str'  for x in colunas if x != "Idade"}

# Lê csv
pacientes = pd.read_csv(r'D:\Banco_integracao\Base pastas separadas\Banco de dados Paciente.csv', sep="\t", decimal=",", encoding = "utf-16", dtype=dict_dtypes).iloc[:-4, :]

In [ ]:
# Passa pacientes e ceps para o db

total = len(pacientes.index)
inicio = datetime.now()
print(f'Início: {inicio}\nTotal: {total}' )
count_pacientes = 0

# Conecta ao db
conn = psycopg2.connect(database="SMSSP", user = "postgres", password = "123", host = "127.0.0.1", port = "5432")
c = conn.cursor()


def pg_pacientes(row):

    global count_pacientes

    # Checa se paciente ja está na base
    c.execute(
        """
        SELECT id FROM pacientes WHERE cns = (%s)
        """,
        (
        row['Código CNS'],
        )
    )

    id_paciente = c.fetchone()
    
    # Se estiver, pega id
    if id_paciente:
        
        id_paciente = id_paciente[0]

    # Se não estiver, insere 
    else:

        c.execute(
            """
            INSERT INTO pacientes (cns, idade, sexo) VALUES (%s, %s, %s)
            RETURNING id
            """,
            (
            row['Código CNS'],
            row['Idade'],
            row['Sexo']
            )            
        )
        id_paciente = c.fetchone()[0]


    # Adiciona entrada paciente-cep se cep é válido
    if len(row['CEP Completo']) == 8:

        # Insere entrada paciente-cep
        c.execute(
            """
            INSERT INTO ceps_pacientes (paciente_id, cep) VALUES (%s, %s)
            """,
            (
            id_paciente,
            row['CEP Completo']
            )            
        )

    # Conta uma iteração
    count_pacientes += 1
    if count_pacientes % 100000 == 0:
        print(f'{count_pacientes} de {total}, tempo: {datetime.now() - inicio}')


pacientes.apply(pg_pacientes, axis=1)

# Salva e fecha
conn.commit()
conn.close()

del pacientes

# Agendamentos

In [ ]:
# Le csv para pegar colunas e especificar tipo de cada
for pasta in ([f.path for f in os.scandir(r'D:\Banco_integracao\Base pastas separadas\AG 67a 2014')] + [r"D:\Banco_integracao\Base pastas separadas\AG-67a 10 2015 CRSCO ok"]):
    for f in os.scandir(pasta):
        df = le_csv(f.path)
        break
    
    colunas = df.columns
    dtypes_ags = {x : 'str'  for x in colunas if x not in ['Número Total Dias Solicitação','Quantidade Agendamento']}
    print(dtypes_ags)

    df = le_csv(f.path, dtypes_ags)    

    for row in df.iterrows():
        print(row)
        print(df.info())
        sys.exit()

In [ ]:
# Lê agendamentos

# Trecho para analisar o progresso durante a execução
files_ag = 0
for pasta in ([f.path for f in os.scandir(r'D:\Banco_integracao\Base pastas separadas\AG 67a 2014')] + [r"D:\Banco_integracao\Base pastas separadas\AG-67a 10 2015 CRSCO ok"]):
    for f in os.scandir(pasta):
        files_ag += 1
inicio = datetime.now()
print(f'Início: {inicio}')
count_ags = 0

# Conecta ao db
conn = psycopg2.connect(database="SMSSP", user = "postgres", password = "123", host = "127.0.0.1", port = "5432")
c = conn.cursor()

def pg_agendamentos(row):
    """
    Insere uma linha do df de agendamentos no banco de dados.
    """

    # Checa se estabelecimento solicitante já está salvo, se não insere. Retorna o id em seguida.
    select_solic = {
       'nome': row['H1 - Nome Estabelecimento Solicitante'],
       'cep': row['CEP']
    }
    insert_solic = {
        'nivel_2': row['H2 - Nome Nível 2'],
        'nivel_3': row['H2 - Nome Nível 3'], 
        'nivel_4': row['H2 - Nome Nível 4'], 
        'nivel_5': row['H2 - Nome Nível 5'], 
        'tipo': row['Tipo Estabelecimento Solicitante'], 
        'nome': row['H1 - Nome Estabelecimento Solicitante'], 
        'cep': row['CEP']
    }
    id_solicitante = check_insert(c, 'estabelecimentos', select_solic, insert_solic)

    # Checa se estabelecimento executante já está salvo, se não insere. Retorna o id em seguida.
    select_exec = {
       'nome': row['H1 - Nome Estabelecimento Executante'],
       'cep': row['Número CEP']
    }
    insert_exec = {
        'nivel_2': row['H2 - Nome Nível 2.1'],
        'nivel_3': row['H2 - Nome Nível 3.1'], 
        'nivel_4': row['H2 - Nome Nível 4.1'], 
        'nivel_5': row['H2 - Nome Nível 5.1'], 
        'tipo': row['Tipo Estabelecimento Executante'], 
        'nome': row['H1 - Nome Estabelecimento Executante'], 
        'cep': row['Número CEP']
    }
    id_executante = check_insert(c, 'estabelecimentos', select_exec, insert_exec)           

    # Checa se procedimento já está salvo, se não insere. Retorna o id em seguida.
    select_proc = {
       'codigo': row['Código Procedimento'],
       'especialidade': row['Nome Especialidade']
    }
    insert_proc = {
        'codigo': row['Código Procedimento'],
        'especialidade': row['Nome Especialidade'],
        'nome': row['Nome Procedimento']
    }
    id_procedimento = check_insert(c, 'procedimentos', select_proc, insert_proc)

    # Checa se paciente já está salvo, se não insere. Retorna o id em seguida.
    select_pac = {
       'cns': row['Código CNS'],
    }
    insert_pac = {
        'cns': row['Código CNS'],
        'idade': None,
        'sexo': None
    }
    id_paciente = check_insert(c, 'pacientes', select_pac, insert_pac)    
    
    # Insere agendamento
    c.execute(
        '''
        INSERT INTO agendamentos (data_vaga, solicitante_id, executante_id, procedimento_id, situacao, data_agen, num_dias, paciente_id, quantidade) 
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
        ''',
        (
        row["Data Vaga"] if row["Data Vaga"] == row["Data Vaga"] else None,
        id_solicitante,
        id_executante,
        id_procedimento,
        row['Nome Situação Agendamento'],
        row["Data Agendamento"] if row["Data Agendamento"] == row["Data Agendamento"] else None,
        row['Número Total Dias Solicitação'],
        id_paciente,
        row['Quantidade Agendamento']
        )
    )


# Le cada csv de agendamentos de 2014 e 2015
for pasta in ([f.path for f in os.scandir(r'D:\Banco_integracao\Base pastas separadas\AG 67a 2014')] + [r"D:\Banco_integracao\Base pastas separadas\AG-67a 10 2015 CRSCO ok"]):
    for f in os.scandir(pasta):
        
        # Progresso
        count_ags += 1
        print(f'{count_ags} de {files_ag}:')

        # Le csv e formata datas
        df = le_csv(f.path, dtypes_ags)
        df.loc[:,['Data Vaga','Data Agendamento']] = df.loc[:,['Data Vaga','Data Agendamento']].apply(pd.to_datetime, errors='coerce')

        # Chama função de inserir agendamento para cada linha
        df.apply(pg_agendamentos, axis = 1)

        print(f'Tempo: {datetime.now() - inicio}')


conn.commit()
conn.close()


# Atendimentos com diagnósticos

In [ ]:
# Le csv para pegar colunas e especificar tipo de cada
for pasta in [r"D:\Banco_integracao\Base pastas separadas\AT 54a 2014", r"D:\Banco_integracao\Base pastas separadas\AT-54a-2015"]:
    for f in os.scandir(pasta):
        df = le_csv(f.path)
        break

    colunas = df.columns
    dtypes_ags = {x: 'str' for x in colunas if x not in ['Quantidade Diagnóstico']}
    print(dtypes_ags)


    df = le_csv(f.path, dtypes_ags)

    for row in df.iterrows():
        print(row)
        print(df.info())
        sys.exit()

In [ ]:
# Trecho para analisar o progresso durante a execução
files_ag = 0
for pasta in [r"D:\Banco_integracao\Base pastas separadas\AT 54a 2014", r"D:\Banco_integracao\Base pastas separadas\AT-54a-2015"]:
    for f in os.scandir(pasta):
        files_ag += 1
inicio = datetime.now()
print(f'Início: {inicio}')
count_ags = 0

# Conecta ao db
conn = psycopg2.connect(database="SMSSP", user="postgres", password="123", host="127.0.0.1", port="5432")
c = conn.cursor()


def pg_atendimentos_diag(row):
    """
    Insere uma linha do df de atendimentos no banco de dados.
    """

    # Checa se estabelecimento executante já está salvo, se não insere. Retorna o id em seguida.
    select_estab = {
        'nome': row['H2 - Nome Estabelecimento Executante'],
        'cep': row['Número CEP']
    }
    insert_estab = {
        'nivel_2': row['H2 - Nome Nível 2'],
        'nivel_3': row['H2 - Nome Nível 3'],
        'nivel_4': row['H2 - Nome Nível 4'],
        'nivel_5': row['H2 - Nome Nível 5'],
        'tipo': row['Tipo Estabelecimento Executante'],
        'nome': row['H2 - Nome Estabelecimento Executante'],
        'cep': row['Número CEP']
    }
    id_estabelecimento = check_insert(c, 'estabelecimentos', select_estab, insert_estab)

    # Checa se tipo de profissional já está salvo, se não insere. Retorna o id em seguida.
    select_prof = {
        'cbo': row['Código CBO Executante'],
        'descricao': row['Nome CBO Executante']
    }
    insert_prof = {
        'cbo': row['Código CBO Executante'],
        'descricao': row['Nome CBO Executante']
    }
    id_profissional = check_insert(c, 'profissionais', select_prof, insert_prof)

    # Checa se diagnostico já está salvo, se não insere. Retorna o id em seguida.
    select_diag = {
        'cid': row['Código CID'],
        'nome': row['Nome CID']
    }
    insert_diag = {
        'cid': row['Código CID'],
        'nome': row['Nome CID']
    }
    id_diagnostico = check_insert(c, 'diagnosticos', select_diag, insert_diag)

    # Checa se paciente já está salvo, se não insere. Retorna o id em seguida.
    select_pac = {
        'cns': row['Código CNS'],
    }
    insert_pac = {
        'cns': row['Código CNS'],
        'idade': None,
        'sexo': None
    }
    id_paciente = check_insert(c, 'pacientes', select_pac, insert_pac)

    # Insere atendimento
    c.execute(
        '''
        INSERT INTO atendimentos (data, profissional_id, diagnostico_id, quant_diag, cid_principal, paciente_id, estabelecimento_id) 
        VALUES (%s, %s, %s, %s, %s, %s, %s)
        ''',
        (
            row["Data Completa"] if row["Data Completa"] == row["Data Completa"] else None,
            id_profissional,
            id_diagnostico,
            row['Quantidade Diagnóstico'],
            row['Indicador de CID Principal'],
            id_paciente,
            id_estabelecimento
        )
    )


# Le cada csv de agendamentos de 2014 e 2015
for pasta in [r"D:\Banco_integracao\Base pastas separadas\AT 54a 2014", r"D:\Banco_integracao\Base pastas separadas\AT-54a-2015"]:
    for f in os.scandir(pasta):

        # if count_ags == 3:
        #     sys.exit()

        # Progresso
        count_ags += 1
        print(f'{count_ags} de {files_ag}:')

        # Le csv e formata datas
        df = le_csv(f.path, dtypes_ags)
        df.loc[:, 'Data Completa'] = df.loc[:, 'Data Completa'].apply(
            pd.to_datetime, errors='coerce')

        # Chama função de inserir agendamento para cada linha
        df.apply(pg_atendimentos_diag, axis=1)

        print(f'Tempo: {datetime.now() - inicio}')

        conn.commit()

        with open('atends diags lidos.txt', 'a') as log_file:
            log_file.write(f.path + '\n')

conn.close()


# Atendimentos com procedimentos

In [ ]:
# Le csv para pegar colunas e especificar tipo de cada
for pasta in [
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2014",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2014 final",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2015",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b ATENDIMENTO BANCO DE DADOS PESQUISA 11_2015 CO1 OK",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b ATENDIMENTO-restante2015"
    ]:
    for f in os.scandir(pasta):
        df = le_csv(f.path)
        break

    colunas = df.columns
    dtypes_ags = {x: 'str' for x in colunas if x not in ['Quantidade Procedimento']}
    print(dtypes_ags)


    df = le_csv(f.path, dtypes_ags)

    for row in df.iterrows():
        print(row)
        print(df.info())
        break
    break

In [ ]:
# Trecho para analisar o progresso durante a execução
files_ag = 0
for pasta in [
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2014",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2014 final",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2015",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b ATENDIMENTO BANCO DE DADOS PESQUISA 11_2015 CO1 OK",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b ATENDIMENTO-restante2015"
    ]:
    for f in os.scandir(pasta):
        files_ag += 1
inicio = datetime.now()
print(f'Início: {inicio}')
count_ags = 0

# Conecta ao db
conn = psycopg2.connect(database="SMSSP", user="postgres", password="123", host="127.0.0.1", port="5432")
c = conn.cursor()


def pg_atendimentos_proc(row):
    """
    Insere uma linha do df de atendimentos no banco de dados.
    """

    # Checa se estabelecimento executante já está salvo, se não insere. Retorna o id em seguida.
    select_estab = {
        'nome': row['H2 - Nome Estabelecimento Executante'],
        'cep': row['Número CEP']
    }
    insert_estab = {
        'nivel_2': row['H2 - Nome Nível 2'],
        'nivel_3': row['H2 - Nome Nível 3'],
        'nivel_4': row['H2 - Nome Nível 4'],
        'nivel_5': row['H2 - Nome Nível 5'],
        'tipo': row['Tipo Estabelecimento Executante'],
        'nome': row['H2 - Nome Estabelecimento Executante'],
        'cep': row['Número CEP']
    }

    id_estabelecimento = check_insert(c, 'estabelecimentos', select_estab, insert_estab)
    

    # Checa se tipo de profissional já está salvo, se não insere. Retorna o id em seguida.
    select_prof = {
        'cbo': row['Código CBO Executante'],
        'descricao': row['Nome CBO Executante']
    }
    insert_prof = {
        'cbo': row['Código CBO Executante'],
        'descricao': row['Nome CBO Executante']
    }
    
    id_profissional = check_insert(c, 'profissionais', select_prof, insert_prof)


     # Checa se procedimento já está salvo, se não insere. Retorna o id em seguida.
    select_proc = {
       'codigo': row['Código Procedimento'],
       'especialidade': row['Nome Especialidade']
    }
    insert_proc = {
        'codigo': row['Código Procedimento'],
        'especialidade': row['Nome Especialidade'],
        'nome': row['Nome Procedimento']
    }
    
    id_procedimento = check_insert(c, 'procedimentos', select_proc, insert_proc)


    # Checa se paciente já está salvo, se não insere. Retorna o id em seguida.
    select_pac = {
        'cns': row['Código CNS'],
    }
    insert_pac = {
        'cns': row['Código CNS'],
        'idade': None,
        'sexo': None
    }
    
    id_paciente = check_insert(c, 'pacientes', select_pac, insert_pac)


    # Insere atendimento
    c.execute(
        '''
        INSERT INTO atendimentos (data, profissional_id, procedimento_id, quant_proc, paciente_id, estabelecimento_id) 
        VALUES (%s, %s, %s, %s, %s, %s)
        ''',
        (
            row["Data Completa"] if row["Data Completa"] == row["Data Completa"] else None,
            id_profissional,
            id_procedimento,
            row['Quantidade Procedimento'] if row['Quantidade Procedimento'] == row['Quantidade Procedimento'] else None,
            id_paciente,
            id_estabelecimento
        )
    )


# Le cada csv de agendamentos de 2014 e 2015
for pasta in [
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2014",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2014 final",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b 2015",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b ATENDIMENTO BANCO DE DADOS PESQUISA 11_2015 CO1 OK",
    r"D:\Banco_integracao\Base pastas separadas\AT-54b ATENDIMENTO-restante2015"
    ]:
    for f in os.scandir(pasta):

        with open('atends procs lidos.txt', 'r') as log_file:
            lidos = log_file.read().splitlines()

        count_ags = len(lidos)

        if f.path in lidos:
            print(f'JÁ LEU: {f.path}')
            continue

        # Progresso
        #count_ags += 1
        print(f'{count_ags} de {files_ag}:')

        # Le csv e formata datas
        df = le_csv(f.path, dtypes_ags)
        df.loc[:, 'Data Completa'] = df.loc[:, 'Data Completa'].apply(
            pd.to_datetime, errors='coerce')

        # Preenche nulls com None
        df.iloc[:, 1:-1] = df.iloc[:, 1:-1].where(df.iloc[:, 1:-1].notnull(), None)        

        # Chama função de inserir atendimento para cada linha
        df.apply(pg_atendimentos_proc, axis=1)

        print(f'Tempo: {datetime.now() - inicio} Marcado em: {datetime.now()}')

        conn.commit()

        with open('atends procs lidos.txt', 'a') as log_file:
            log_file.write(f.path + '\n')

conn.close()
